- Fine-tuning ParsBERT on PersianQA and ParSQUAD Datasets 

[Model and Datasets Path](https://drive.google.com/drive/u/1/folders/11sljYfapRzROufDcvlJ_lKrQmk6GNc_f)

# Setup

In [2]:
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.1 MB/s eta 0:00:00


In [3]:
import json
import pandas as pd

In [4]:
from datasets import load_dataset, load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 3e-5
epoch = 2
DRIVE_PATH = "/content/drive/MyDrive/project/"


In [ ]:
!nvidia-smi

Fri May  5 20:22:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load and Process Data

In [15]:
train_path = DRIVE_PATH + 'dataset/train_2.csv'
test_path  = DRIVE_PATH + 'dataset/validation_2.csv'


In [16]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [17]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [18]:
train_dataset.save_to_disk("train.hf")
test_dataset.save_to_disk("test.hf")

Saving the dataset (0/1 shards):   0%|          | 0/49573 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3235 [00:00<?, ? examples/s]

In [20]:
!cp -r '/content/train.hf' "/content/drive/MyDrive/project/dataset/train.hf"
!cp -r '/content/test.hf' "/content/drive/MyDrive/project/dataset/test.hf"


cp: cannot create directory '/content/drive/MyDrive/project/dataset/train.hf/train.hf': Read-only file system
cp: cannot create directory '/content/drive/MyDrive/project/dataset/test.hf/test.hf': Read-only file system


# Load Datasets

In [21]:
train_dataset = load_from_disk('/content/train.hf' ).shuffle(seed=42)
test_dataset = load_from_disk('/content/test.hf').shuffle(seed=42)

In [ ]:
print(len(train_dataset))

49573


In [ ]:
train_dataset[1]['answer_start']

353.0

# Tokenize Datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if examples["answer_start"][sample_index] == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = examples["answer_start"][sample_index]
            end_char = start_char + len(examples["answers"][sample_index])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
# the datasets library does cashing itself, batched is multitreading for fast-tokenizer
tokenized_train = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_test = test_dataset.map(prepare_train_features, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/49573 [00:00<?, ? examples/s]

Map:   0%|          | 0/49573 [00:00<?, ? examples/s]

# Training 

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized 

In [ ]:
args = TrainingArguments(
    DRIVE_PATH + f"Model Checkpoints",
    save_strategy = 'epoch',
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001) 

In [ ]:
trainer = Trainer(
    model=model.cuda(),
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer)

In [ ]:
# start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.363300,0.802748


Epoch,Training Loss,Validation Loss
1,1.363300,0.802748
2,0.801600,0.439078


TrainOutput(global_step=12404, training_loss=1.2270955587348336, metrics={'train_runtime': 7360.2078, 'train_samples_per_second': 13.481, 'train_steps_per_second': 1.685, 'total_flos': 1.3740035801039016e+16, 'train_loss': 1.2270955587348336, 'epoch': 2.0})

# Test Model

In [8]:
!pip install -q sentencepiece

from tqdm import tqdm
from IPython.display import clear_output
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer


In [9]:
DRIVE_PATH = "/content/drive/MyDrive/project/"
model_path = DRIVE_PATH + f"Model Checkpoints/checkpoint-12404/"  ## load model trained for 2 epochs
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [10]:
class AnswerPredictor:
  def __init__(self, model, tokenizer, device='cuda', n_best=10, max_length=512, stride=256, no_answer=False):
      """Initializes PyTorch Question Answering Prediction
      It's best to leave use the default values.
      Args:
          model: Fine-tuned torch model
          tokenizer: Transformers tokenizer
          device (torch.device): Running device
          n_best (int): Number of best possible answers
          max_length (int): Tokenizer max length
          stride (int): Tokenizer stride
          no_answer (bool): If True, model can return "no answer"
      """
      self.model = model.eval().to(device)
      self.tokenizer = tokenizer
      self.device = device
      self.max_length = max_length
      self.stride = stride
      self.no_answer = no_answer
      self.n_best = n_best


  def model_pred(self, questions, contexts, batch_size=1):
      n = len(contexts)
      if n%batch_size!=0:
          raise Exception("batch_size must be divisible by sample length")

      tokens = self.tokenizer(questions, contexts, add_special_tokens=True, 
                              return_token_type_ids=True, return_tensors="pt", padding=True, 
                              return_offsets_mapping=True, truncation="only_second", 
                              max_length=self.max_length, stride=self.stride)

      start_logits, end_logits = [], []
      for i in tqdm(range(0, n-batch_size+1, batch_size)):
          with torch.no_grad():
              out = self.model(tokens['input_ids'][i:i+batch_size].to(self.device), 
                          tokens['attention_mask'][i:i+batch_size].to(self.device), 
                          tokens['token_type_ids'][i:i+batch_size].to(self.device))

              start_logits.append(out.start_logits)
              end_logits.append(out.end_logits)

      return tokens, torch.stack(start_logits).view(n, -1), torch.stack(end_logits).view(n, -1)


  def __call__(self, questions, contexts, batch_size=1, answer_max_len=100):
      """Creates model prediction
      
      Args: 
          questions (list): Question strings
          contexts (list): Contexts strings
          batch_size (int): Batch size
          answer_max_len (int): Sets the longests possible length for any answer
        
      Returns:
          dict: The best prediction of the model
              (e.g {0: {"text": str, "score": int}})
      """
      questions = [questions]
      contexts = [contexts]
      tokens, starts, ends = self.model_pred(questions, contexts, batch_size=batch_size)
      start_indexes = starts.argsort(dim=-1, descending=True)[:, :self.n_best]
      end_indexes = ends.argsort(dim=-1, descending=True)[:, :self.n_best]

      preds = {}
      for i, (c, q) in enumerate(zip(contexts, questions)):  
          min_null_score = starts[i][0] + ends[i][0] # 0 is CLS Token
          start_context = tokens['input_ids'][i].tolist().index(self.tokenizer.sep_token_id)
          
          offset = tokens['offset_mapping'][i]
          valid_answers = []
          for start_index in start_indexes[i]:
              # Don't consider answers that are in questions
              if start_index<start_context:
                  continue
              for end_index in end_indexes[i]:
                  # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                  # to part of the input_ids that are not in the context.
                  if (start_index >= len(offset) or end_index >= len(offset)
                      or offset[start_index] is None or offset[end_index] is None):
                      continue
                  # Don't consider answers with a length that is either < 0 or > max_answer_length.
                  if end_index < start_index or (end_index-start_index+1) > answer_max_len:
                      continue

                  start_char = offset[start_index][0]
                  end_char = offset[end_index][1]
                  valid_answers.append({"score": (starts[i][start_index] + ends[i][end_index]).item(),
                                        "text": c[start_char: end_char]})
                  
          if len(valid_answers) > 0:
              best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
          else:
              best_answer = {"text": "", "score": min_null_score}

          if self.no_answer:
              preds[i] = best_answer if best_answer["score"] >= min_null_score else {"text": "", "score": min_null_score}
          else:
              preds[i] = best_answer

      return preds


In [11]:
predictor = AnswerPredictor(model, tokenizer, device='cuda', n_best=10, no_answer=True)


In [ ]:
test_dataset[9]

{'Unnamed: 0': 39900,
 'title': 'ضبط گرامافون',
 'context': 'سطح سر و صدای سطح پایین وینیل نسبت به شلاک فراموش نشد و همچنین دوام آن فراموش نشد. در اواخر دهه 1930 ، تبلیغات رادیویی و برنامه های رادیویی از قبل ضبط شده برای سوار بر دیسک ها شروع به مهر زدن در وینیل می کردند ، بنابراین از طریق پست نمی شکنند. در اواسط دهه 1940 ، به همین دلیل ، نسخه های ویژه دی وی دی ضبط شده از وینیل نیز شروع شد. اینها همه دور در دقیقه 78 بودند. در طول جنگ جهانی دوم و بعد از آن ، هنگامی که منابع شلاک بسیار محدود بود ، برخی از رکوردهای 78 دور در دقیقه به جای شلاک ، در وینیل تحت فشار قرار گرفتند ، به ویژه رکوردهای شش دقیقه ای 12 اینچ (30 سانتی متر) 78 دور در دقیقه که توسط V-Disc تولید شده و برای توزیع به یونایتد تولید شده است. سربازان ایالات متحده در جنگ جهانی دوم. در دهه 1940 ، رونویسی رادیو که معمولاً در رکوردهای 16 اینچی بود ، اما گاهی 12 اینچ بود ، همیشه از وینیل ساخته می شد ، اما در 33 ⁄ 1 دور در دقیقه بریده می شد. رونویسی های کوتاهتر اغلب در 78 دور در دقیقه بریده می شدند.',
 'question': 'کدام ماده با دوام

In [ ]:
context = test_dataset[9]['context'] 
question = test_dataset[9]['question'] 
answer = test_dataset[9]['answers']
print(answer)


preds = predictor(question, context, batch_size=1)
pred = preds[0]['text'].strip()
print('\n\n Model Prediction: ', preds[0]['text'].strip())
print(preds)

# print(compute_exact_match(answer, pred))
# print(compute_f1(answer, pred))

وینیل


100%|██████████| 1/1 [00:00<00:00,  2.21it/s]



 Model Prediction:  وینیل
{0: {'score': -1.2323240041732788, 'text': 'وینیل'}}


# Evaluation

In [12]:
from nltk.translate.bleu_score import sentence_bleu


def compute_f1(prediction, answer):
    pred_tokens = prediction.split()
    answer_tokens = answer.split()
    
    if len(pred_tokens) == 0 or len(answer_tokens) == 0:
        return int(pred_tokens == answer_tokens)
    
    common_tokens = set(pred_tokens) & set(answer_tokens)
    
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(answer_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(answer_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_exact_match(prediction, answer):
    return int(prediction == answer)

def bleu(prediction, answer) : 
  reference = [answer.split(' ')]
  candidate = pred.split(' ')
  BLEU = sentence_bleu(reference, candidate)
  BLEU1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
  BLEU4 = sentence_bleu(reference, candidate, weights=(0, 0, 0, 1))

  return BLEU, BLEU1, BLEU4


In [22]:
predictor = AnswerPredictor(model, tokenizer, device='cuda', n_best=10, no_answer=True)

EH = 0
F1 = 0
BLEU = 0
BLEU1 = 0
BLEU4 = 0
for example in test_dataset: 

  #print(len(example['answers']))
  #if len(example['answers']) != 2: 
    #print(example['answers'])
  if example['answers'] == [] :
    context = example['context'] 
    question = example['question'] 
    preds = predictor([question], [context], batch_size=1)
    pred = preds[0]['text'].strip()
    if pred == [] : 
      EH += 1
      F1 += 1

    continue

  context = example['context'] 
  question = example['question'] 
  answer = example['answers']
  preds = predictor(question, context, batch_size=1)
  pred = preds[0]['text'].strip()

  EH += compute_exact_match(pred, answer)
  F1 += compute_f1(pred, answer)
  b, b1, b4 = bleu(pred, answer)
  BLEU += b
  BLEU1 += b1
  BLEU4 += b4

EH /= len(test_dataset)
F1 /= len(test_dataset)
BLEU /= len(test_dataset)
BLEU1 /= len(test_dataset)
BLEU4 /= len(test_dataset)

100%|██████████| 1/1 [00:00<00:00, 71.63it/s]
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram orde

In [24]:
print('Exact match of ParsBERT on testset : ' + str(EH))
print('F1 score of ParsBERT on testset : ' + str(F1))
# print('BLEU score of ParsBERT on testset : ' + str(BLEU))
print('BLEU1 score of ParsBERT on testset : ' + str(BLEU1))
# print('BLEU4 score of ParsBERT on testset : ' + str(BLEU4))
     


Exact match of ParsBERT on testset : 0.5468315301391036
F1 score of ParsBERT on testset : 0.7075234576747764
BLEU1 score of ParsBERT on testset : 0.6642866278267102
